<a href="https://colab.research.google.com/github/Cayo-Cezar/spark-imdb/blob/main/spark_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null


!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

!ls -l spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz

!pip install -q pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
-rw-r--r-- 1 root root 388341449 Jun 19  2023 spark-3.4.1-bin-hadoop3.tgz


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Teste Spark").getOrCreate()

spark


In [ ]:
from google.colab import files
arquivo = files.upload()

Saving MOVIES.csv to MOVIES.csv


In [ ]:

df = spark.read.csv('MOVIES.csv',header=True, inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: string (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: string (nullable = true)
 |-- Gross: string (nullable = true)



In [ ]:
df.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [ ]:

# 1. Estatísticas Descritivas das Colunas Numéricas
# Para colunas como 'IMDB_Rating' e 'Meta_score'
df.select("IMDB_Rating", "Meta_score", "No_of_Votes").describe().show()

+-------+------------------+--------------------+-------------------+
|summary|       IMDB_Rating|          Meta_score|        No_of_Votes|
+-------+------------------+--------------------+-------------------+
|  count|              1000|                 844|               1000|
|   mean| 7.949300000000012|   78.03734939759036| 272714.91277890466|
| stddev|0.2754912074920139|  12.363212589581746| 326898.37046629854|
|    min|               7.6| Jr. finds himsel...|            1000639|
|    max|               9.3|                  99|Wotan Wilke Möhring|
+-------+------------------+--------------------+-------------------+



In [ ]:
top_rated_movies = df.orderBy(df.IMDB_Rating.desc()).select("Series_Title", "IMDB_Rating", "Director").limit(10)
print("Top 10 filmes com melhores avaliações:")
top_rated_movies.show()

Top 10 filmes com melhores avaliações:
+--------------------+-----------+--------------------+
|        Series_Title|IMDB_Rating|            Director|
+--------------------+-----------+--------------------+
|The Shawshank Red...|        9.3|      Frank Darabont|
|       The Godfather|        9.2|Francis Ford Coppola|
|     The Dark Knight|        9.0|   Christopher Nolan|
|The Godfather: Pa...|        9.0|Francis Ford Coppola|
|        12 Angry Men|        9.0|        Sidney Lumet|
|The Lord of the R...|        8.9|       Peter Jackson|
|        Pulp Fiction|        8.9|   Quentin Tarantino|
|    Schindler's List|        8.9|    Steven Spielberg|
|           Inception|        8.8|   Christopher Nolan|
|          Fight Club|        8.8|       David Fincher|
+--------------------+-----------+--------------------+



In [ ]:
# 3. Filmes mais populares com base no número de votos

from pyspark.sql.functions import col

# Corrigir a ordenação com base nos votos (No_of_Votes) após a conversão
most_voted_movies = df_converted \
    .orderBy(col("No_of_Votes").desc()) \
    .select("Series_Title", "No_of_Votes", "IMDB_Rating") \
    .limit(10)

# Exibir os 10 filmes com mais votos
print("Top 10 filmes com mais votos:")
most_voted_movies.show()


Top 10 filmes com mais votos:
+--------------------+-----------+-----------+
|        Series_Title|No_of_Votes|IMDB_Rating|
+--------------------+-----------+-----------+
|The Shawshank Red...|    2343110|        9.3|
|     The Dark Knight|    2303232|        9.0|
|           Inception|    2067042|        8.8|
|          Fight Club|    1854740|        8.8|
|        Pulp Fiction|    1826188|        8.9|
|        Forrest Gump|    1809221|        8.8|
|          The Matrix|    1676426|        8.7|
|The Lord of the R...|    1661481|        8.8|
|The Lord of the R...|    1642758|        8.9|
|       The Godfather|    1620367|        9.2|
+--------------------+-----------+-----------+



In [ ]:
# 4. Contagem de Filmes por Gênero

from pyspark.sql.functions import explode, split

genre_df = df.withColumn("Genre", explode(split(df.Genre, ", ")))
genre_counts = genre_df.groupBy("Genre").count().orderBy("count", ascending=False)

print("Contagem de filmes por gênero:")
genre_counts.show()

Contagem de filmes por gênero:
+---------+-----+
|    Genre|count|
+---------+-----+
|    Drama|  724|
|   Comedy|  233|
|    Crime|  209|
|Adventure|  196|
|   Action|  189|
| Thriller|  137|
|  Romance|  125|
|Biography|  109|
|  Mystery|   99|
|Animation|   82|
|   Sci-Fi|   67|
|  Fantasy|   66|
|   Family|   56|
|  History|   56|
|      War|   51|
|    Music|   35|
|   Horror|   32|
|  Western|   20|
|Film-Noir|   19|
|    Sport|   19|
+---------+-----+
only showing top 20 rows



--------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import col, regexp_replace, explode, split

# 1. Limpeza de caracteres e conversão de tipos de dados
df_converted = df \
    .withColumn("No_of_Votes", regexp_replace("No_of_Votes", ",", "").cast("integer")) \
    .withColumn("Gross", regexp_replace("Gross", "[$,]", "").cast("double")) \
    .withColumn("Released_Year", col("Released_Year").cast("integer")) \
    .withColumn("Meta_score", col("Meta_score").cast("integer"))


In [ ]:
# 2. Remover linhas com valores nulos em colunas principais, após conversão
df_converted = df_converted.na.drop(subset=["No_of_Votes", "Gross", "IMDB_Rating"])

In [ ]:
# 3. Estatísticas Descritivas para 'No_of_Votes' e 'Gross'
print("Estatísticas descritivas para 'No_of_Votes' e 'Gross':")
df_converted.select("No_of_Votes", "Gross").describe().show()

Estatísticas descritivas para 'No_of_Votes' e 'Gross':
+-------+------------------+--------------------+
|summary|       No_of_Votes|               Gross|
+-------+------------------+--------------------+
|  count|               817|                 817|
|   mean|314781.67441860464| 6.738834272949816E7|
| stddev| 343472.9559634733|1.0921154318279986E8|
|    min|             25088|              1305.0|
|    max|           2343110|        9.36662225E8|
+-------+------------------+--------------------+



In [ ]:
# 4. Top 10 Filmes com a Maior Bilheteria (Gross)
top_grossing_movies = df_converted.orderBy(col("Gross").desc()).select("Series_Title", "Gross", "IMDB_Rating").limit(10)
print("Top 10 filmes com a maior bilheteria:")
top_grossing_movies.show()

Top 10 filmes com a maior bilheteria:
+--------------------+------------+-----------+
|        Series_Title|       Gross|IMDB_Rating|
+--------------------+------------+-----------+
|Star Wars: Episod...|9.36662225E8|        7.9|
|   Avengers: Endgame|   8.58373E8|        8.4|
|              Avatar|7.60507625E8|        7.8|
|Avengers: Infinit...|6.78815482E8|        8.4|
|             Titanic|6.59325379E8|        7.8|
|        The Avengers|6.23279547E8|        8.0|
|       Incredibles 2|6.08581744E8|        7.6|
|     The Dark Knight|5.34858444E8|        9.0|
|           Rogue One|5.32177324E8|        7.8|
|The Dark Knight R...|4.48139099E8|        8.4|
+--------------------+------------+-----------+



In [ ]:
# 5. Média de votos por gênero
# Dividir a coluna de gêneros e contar a média de votos
genre_df = df_converted.withColumn("Genre", explode(split(col("Genre"), ", ")))
average_votes_per_genre = genre_df.groupBy("Genre").avg("No_of_Votes").orderBy("avg(No_of_Votes)", ascending=False)

In [ ]:
print("Média de votos por gênero:")
average_votes_per_genre.show()

Média de votos por gênero:
+---------+------------------+
|    Genre|  avg(No_of_Votes)|
+---------+------------------+
|   Sci-Fi| 598005.6333333333|
|Adventure|479657.58181818185|
|   Action|473315.70666666667|
|  Fantasy|          378631.2|
| Thriller| 367590.5046728972|
|  Mystery|334111.59523809527|
|    Crime|329696.20238095237|
|Animation|309274.56923076924|
|   Horror| 286840.2272727273|
|    Drama|276654.48900169204|
|Biography| 273541.2916666667|
|  Western|       272667.3125|
|   Comedy|  261025.743315508|
|      War|242400.83783783784|
|   Family|240716.91666666666|
|  History|235024.06976744186|
|    Sport|230174.16666666666|
|  Romance| 223521.7663551402|
|    Music|       149388.9375|
|Film-Noir| 93989.09090909091|
+---------+------------------+
only showing top 20 rows



In [ ]:
# 6. Ano com o maior número de votos totais
yearly_votes = df_converted.groupBy("Released_Year").sum("No_of_Votes").orderBy(col("sum(No_of_Votes)").desc())
print("Ano com o maior número de votos totais:")
yearly_votes.show()

Ano com o maior número de votos totais:
+-------------+----------------+
|Released_Year|sum(No_of_Votes)|
+-------------+----------------+
|         2014|        12816155|
|         2009|        10116854|
|         2001|         9985829|
|         2010|         9892494|
|         2012|         9433276|
|         1999|         9128338|
|         2004|         9107806|
|         2008|         8878415|
|         2006|         8767012|
|         1994|         8615618|
|         2013|         7614274|
|         2015|         7232173|
|         1995|         7174701|
|         2003|         6834292|
|         2007|         6753906|
|         2000|         6677095|
|         2016|         6334307|
|         2002|         6304634|
|         2011|         6109071|
|         1997|         5635223|
+-------------+----------------+
only showing top 20 rows

